In [1]:
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2023-06-01 19:30:11.023814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
model = TFAutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")


In [24]:
text = "Hola mundo"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
output[1]

(<tf.Tensor: shape=(2, 1, 12, 2, 64), dtype=float32, numpy=
 array([[[[[-5.38292527e-01,  2.48257160e-01, -5.22796094e-01, ...,
            -1.59949288e-01,  1.91841722e-02, -5.79061687e-01],
           [ 1.38699219e-01,  3.85100216e-01, -7.28040785e-02, ...,
             4.47283864e-01,  5.61189465e-02, -3.39824677e-01]],
 
          [[ 1.38657653e+00,  2.03933507e-01,  7.05375969e-01, ...,
             4.38653916e-01, -5.71687579e-01,  3.57128829e-01],
           [ 1.21856704e-02,  1.00735441e-01, -2.47440279e-01, ...,
            -7.77103603e-01,  7.35555053e-01,  2.79312193e-01]],
 
          [[-7.03806758e-01,  1.09546900e+00, -1.15335071e+00, ...,
            -4.36316460e-01, -7.29311705e-01, -3.45411867e-01],
           [ 2.81185806e-02,  4.98431563e-01, -1.37484014e-01, ...,
             4.44768310e-01,  9.50722992e-02,  9.51809168e-01]],
 
          ...,
 
          [[-1.44997388e-01,  1.06378722e+00, -4.78014983e-02, ...,
            -2.81589366e-02,  2.32391089e-01,  2.01418

In [13]:
encoded_input

{'input_ids': <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[3041, 5372]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[1, 1]], dtype=int32)>}

In [11]:
preprocessor = hub.load( "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

# Step 1: tokenize batches of text inputs.
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string) # This SavedModel accepts up to 2 text inputs.
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(text_input)]

# Step 2: pack input sequences for the Transformer encoder.
seq_length = 200  # Your choice here.
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=seq_length))  # Optional argument.
encoder_inputs = bert_pack_inputs(tokenized_inputs)

# Model
encoder = hub.KerasLayer( "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].

In [23]:
embedding_model = tf.keras.Model(text_input, sequence_output)
sentences = tf.constant(["Hello world"])

res = embedding_model(sentences)